In [ ]:
import cv2
import numpy as np
import os

# Method 1: Set environment variables for Qt scaling (recommended)
# These should ideally be set before importing cv2, or in your shell profile
os.environ['QT_SCALE_FACTOR'] = '1'  # Scale factor (2 for 200%, 3 for 300%)
os.environ['QT_AUTO_SCREEN_SCALE_FACTOR'] = '1'

class EdgeDetectorApp:
    def __init__(self, image_path='image.png'):
        self.image: cv2.typing.MatLike = cv2.imread(image_path) # type:ignore
        assert self.image is not None, f"Image file '{image_path}' not found. Please provide a valid image path."
        crops_vertical = [300, 900]
        crops_horizontal = [0, 0]
        self.image = self.image[crops_vertical[0]:self.image.shape[0] - crops_vertical[1],
                               crops_horizontal[0]:self.image.shape[1] - crops_horizontal[1]]
        
        # Convert to grayscale for edge detection
        self.gray = cv2.cvtColor(self.image, cv2.COLOR_BGR2GRAY)
        
        # Initial threshold values
        self.low_threshold = 100
        self.high_threshold = 150

        self.setup_window()
    
    def setup_window(self):
        """Setup the OpenCV window with scaled trackbars"""
        window_name = 'Interactive Edges'

        # Create window
        cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)
        
        # Method 2: Resize the window to be larger
        # This makes the trackbars more usable on high-DPI displays
        cv2.resizeWindow(window_name, width=2160, height=1500)
        
        # Create trackbars
        cv2.createTrackbar('Low Threshold', window_name, self.low_threshold, 255, self.update_edges)
        cv2.createTrackbar('High Threshold', window_name, self.high_threshold, 255, self.update_edges)
        
        
        # Initial edge detection
        self.update_edges(0)
        
    def update_edges(self, val):
        """Callback function for trackbar changes"""
        window_name = 'Interactive Edges'
        # Get current trackbar values
        self.low_threshold = cv2.getTrackbarPos('Low Threshold', window_name)
        if cv2.getTrackbarPos('High Threshold', window_name) > 0:
            self.high_threshold = cv2.getTrackbarPos('High Threshold', window_name)
        # Ensure high threshold is always >= low threshold
        
        # Apply Canny edge detection
        edges = cv2.Canny(self.gray, self.low_threshold, self.high_threshold)

        # Ensure both images are valid numpy arrays before stacking
        if self.image is not None and edges is not None:
            comparison = np.hstack([self.gray, edges])
            # Display the result
            cv2.imshow(window_name, comparison)
        else:
            print("Error: One of the images is None and cannot be stacked.")
    
    def run(self):
        """Main application loop"""
        print("Edge Detection Controls:")
        print("- Use the sliders to adjust Canny edge detection thresholds")
        print("- Press 'q' to quit")
        print("- Press 's' to save the current edge image")
        
        while True:
            key = cv2.waitKey(1)
            
            if key == ord('q'):
                break
            elif key == ord('s'):
                # Save current edge detection result
                edges = cv2.Canny(self.gray, self.low_threshold, self.high_threshold)
                cv2.imwrite('edges_output.png', edges)
                print("Edge image saved as 'edges_output.png'")
        
        cv2.destroyAllWindows()

print("OpenCV Edge Detection with Scaling Solutions")
print("=" * 50)

# Method 1: Use environment variables (recommended)
print("Starting OpenCV version with Qt scaling...")
app = EdgeDetectorApp('image.png')
app.run()

In [2]:
import cv2
import numpy as np
# import os

def show_image(image: cv2.typing.MatLike, title: str = "Image"):
    """Display an image in a window."""
    cv2.imshow(title, image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

image: cv2.typing.MatLike = cv2.imread(
    "../images/Screenshot_20250720_131146_Nonogram_galaxy.png"
)  # type:ignore
crops_vertical = [300, 900]
crops_horizontal = [0, 0]
image = image[
    crops_vertical[0] : image.shape[0] - crops_vertical[1],
    crops_horizontal[0] : image.shape[1] - crops_horizontal[1],
]

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
# turn the image into binary black and white with thresholding
_, binary = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
show_image(binary, "Binary Image")

# low_threshold = 100
# high_threshold = 150
# edges = cv2.Canny(gray, low_threshold, high_threshold)
# show_image(edges, "Edges")

# lines = cv2.HoughLinesP(
#     edges, rho=1, theta=np.pi/180, threshold=100,
#     minLineLength=50, maxLineGap=10
# )

# # Create a copy of the original image to draw lines on
# lines_image = image.copy()

# # Draw the detected lines
# if lines is not None:
#     print(f"Found {len(lines)} lines")
#     for line in lines:
#         x1, y1, x2, y2 = line[0]
#         # Draw line in green color with thickness 2
#         cv2.line(lines_image, (x1, y1), (x2, y2), (0, 255, 0), 2)
    
#     # Display the image with lines
#     show_image(lines_image, "Detected Lines")
    
#     # Separate horizontal and vertical lines for analysis
#     horizontal_lines = []
#     vertical_lines = []
    
#     for line in lines:
#         x1, y1, x2, y2 = line[0]
#         # Calculate angle to determine if line is horizontal or vertical
#         angle = np.arctan2(y2 - y1, x2 - x1) * 180 / np.pi
        
#         # Lines within 10 degrees of horizontal (0 or 180 degrees)
#         if abs(angle) < 10 or abs(angle - 180) < 10:
#             horizontal_lines.append(line)
#         # Lines within 10 degrees of vertical (90 or -90 degrees)
#         elif abs(angle - 90) < 10 or abs(angle + 90) < 10:
#             vertical_lines.append(line)
    
#     print(f"Horizontal lines: {len(horizontal_lines)}")
#     print(f"Vertical lines: {len(vertical_lines)}")
    
#     # Create separate visualizations for horizontal and vertical lines
#     horizontal_image = image.copy()
#     vertical_image = image.copy()
    
#     # Draw horizontal lines in red
#     for line in horizontal_lines:
#         x1, y1, x2, y2 = line[0]
#         cv2.line(horizontal_image, (x1, y1), (x2, y2), (0, 0, 255), 2)
    
#     # Draw vertical lines in blue
#     for line in vertical_lines:
#         x1, y1, x2, y2 = line[0]
#         cv2.line(vertical_image, (x1, y1), (x2, y2), (255, 0, 0), 2)
    
#     show_image(horizontal_image, "Horizontal Lines (Red)")
#     show_image(vertical_image, "Vertical Lines (Blue)")
    
# else:
#     print("No lines detected. Try adjusting the Hough transform parameters.")

In [ ]:
# horizontal_lines = sorted([line for line in lines if abs(line[0][1] - line[0][3]) < 10], key=lambda x: x[0][1])
# vertical_lines   = sorted([line for line in lines if abs(line[0][0] - line[0][2]) < 10], key=lambda x: x[0][0])

# num_rows = len(horizontal_lines) - 1
# num_cols = len(vertical_lines) - 1

# top_labels_img = image[0:grid_top_y, grid_left_x:grid_right_x]
# left_labels_img = image[grid_top_y:grid_bottom_y, 0:grid_left_x]

# import pytesseract

# # Optional: preprocess label regions (e.g. threshold)
# gray_top = cv2.cvtColor(top_labels_img, cv2.COLOR_BGR2GRAY)
# _, binary_top = cv2.threshold(gray_top, 150, 255, cv2.THRESH_BINARY_INV)

# text = pytesseract.image_to_string(binary_top, config='--psm 6 digits')
# print(text)